In [3]:
import datetime
import re
import xarray as xr
import ray
import numpy as np
import pangaea as pa

from gazar.grid import ArrayGrid
from osgeo import osr, gdalconst
from pathlib import Path

In [4]:
WRFOUT_REGEX = r"wrfout_(?P<param>[A-Z])_[a-z0-9]{3,4}_(?P<timestamp>\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2})"
cuencas_api_dict = {'meta': {}, 'csv': {'ppn_acum_diario': {}, 'ppn_por_hora': {}}}

In [5]:
path_wrf_out = f'/home/sagus/Development/temp/wrfout_A_d01_2020-02-06_18:00:00'

In [6]:
def corregir_wrfout(ruta_wrfout):
    """Fixes variables dimensions
        Parameters
        ----------
        ruta_wrfout: str
            route to the nc file to fix
        Returns
        -------
        rundate

    """
    xds = xr.open_dataset(ruta_wrfout)
    variables = ['XLAT', 'XLONG', 'XLAT_U', 'XLONG_U', 'XLAT_V', 'XLONG_V']
    if len(xds.coords['XLAT'].shape) > 2:
        for var in variables:
            xds.coords[var] = xds.coords[var].mean(axis=0)
    rundate = datetime.datetime.strptime(xds.START_DATE, '%Y-%m-%d_%H:%M:%S')

    return rundate, xds

In [24]:
def to_projection(_plsm, variable, projection=None):
    """Convert Grid to New Projection.
        Parameters
        ----------
        variable: :obj:`str`
            Name of variable in dataset.
        projection: :func:`osr.SpatialReference`
            Projection to convert data to.
        Returns
        -------
        :func:`xarray.Dataset`
    """
    if projection is None:

        projection = osr.SpatialReference()
        projection.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 \
                                    +no_defs")
    new_data = []
    for band in range(_plsm._obj.dims[_plsm.dims['Time']]):
        ar_gd = ArrayGrid(in_array=_plsm._obj[variable][band].values[::-1, :],
                          wkt_projection=_plsm.projection.ExportToWkt(),
                          geotransform=_plsm.geotransform)
        ggrid = ar_gd.to_projection(projection, gdalconst.GRA_Average)
        new_data.append(ggrid.np_array())

    _plsm.to_datetime()
    return _plsm.export_dataset(variable, np.array(new_data), ggrid)

In [8]:
def convertir_variable(plsm, variable):
    vari = to_projection(plsm.lsm, variable)
    vari['lat'] = vari['lat'].sel(x=1)
    vari['lon'] = vari['lon'].sel(y=1)
    vari = vari.rename({'lat': 'y', 'lon': 'x'})
    return vari

In [9]:
def guardar_tif(vari, arr, out_path):
    import rasterio
    nw_ds = rasterio.open(out_path, 'w', driver='GTiff',
                          height=arr.shape[0],
                          width=arr.shape[1],
                          count=1, dtype=str(arr.dtype),
                          crs=vari.lsm.projection.ExportToWkt(),
                          transform=vari.lsm.affine)
    nw_ds.write(arr, 1)
    nw_ds.close()

In [10]:
def genear_tif_prec(plsm, out_path=None):
    if out_path is None:
        out_path = 'geotiff/ppn_' + plsm.START_DATE[:-6]
    plsm.variables['RAINNC'].values = plsm.variables['RAINNC'].values + 1000
    plsm.variables['RAINC'].values = plsm.variables['RAINC'].values + 1000
    rainnc = convertir_variable(plsm, 'RAINNC')
    rainc = convertir_variable(plsm, 'RAINC')
    arrs = {}
    for t in range(len(plsm.coords['Time'])):
        arrs[t] = rainnc.RAINNC[t].values[:, :] + rainc.RAINC[t].values[:, :]
        arrs[t][arrs[t] == 0] = np.nan
    for t in range(1, len(plsm.coords['Time'])):
        guardar_tif(rainnc, arrs[t] - arrs[t-1], out_path + '_' + str(t)
                                                          + '.tif')
    guardar_tif(rainnc, arrs[33] - arrs[9], out_path + '.tif')
    plsm.close()

NameError: name 'xds' is not defined

In [11]:
rundate, xds = corregir_wrfout(path_wrf_out)

In [25]:
genear_tif_prec(xds, out_path='geotiff/ppn')

AttributeError: 'LSMGridReader' object has no attribute 'dims'

In [27]:
xds.lsm.time_dim

'time'

In [30]:
xds.lsm._obj.dims[xds.lsm.time_dim]

KeyError: 'time'